In [1]:
import pandas as pd
import numpy as np

# --- 1. MOCK DATA SETUP ---

# Data represents Annual CO2e (tonnes) from core supply chain operations (Scope 3)
# for three hypothetical e-commerce brands over three historical years.
data = {
    'Year': [2022, 2023, 2024],
    'EcoThread_Apparel_CO2e': [1500, 1650, 1850],  # Growing fashion brand
    'GlowUp_Cosmetics_CO2e': [800, 780, 750],      # Stable brand with early reduction efforts
    'TerraHome_Goods_CO2e': [2200, 2450, 2800]     # Fast-growing home goods brand
}
df = pd.DataFrame(data)
df.set_index('Year', inplace=True)

# --- 2. EMISSION FACTORS (FE) AND DRIVERS ---

# Simplified Emission Factors (FE) for demonstration.
# In a real project, this would be highly detailed by material/transport mode.
EMISSION_FACTORS = {
    'Product_Unit_Avg_FE': 20.0,       # kg CO2e per average product unit
    'Packaging_Mass_FE': 2.5,          # kg CO2e per kg of mixed packaging
    'Delivery_FE': 0.8                 # kg CO2e per last-mile parcel
}

print("--- INITIAL DATA OVERVIEW (tonnes CO2e) ---")
print(df)
print("\n--- SIMPLIFIED EMISSION FACTORS USED ---")
for key, value in EMISSION_FACTORS.items():
    print(f"- {key}: {value} kg CO2e")
print("----------------------------------------\n")

# --- 3. HISTORICAL ANALYSIS ---

print("--- HISTORICAL PERFORMANCE (2022-2024) ---")

# Calculate Compound Annual Growth Rate (CAGR) for emissions for prediction
years_diff = df.index.max() - df.index.min()
cagr_results = {}

for brand in df.columns:
    start_emissions = df[brand].iloc[0]
    end_emissions = df[brand].iloc[-1]
    
    # Calculate CAGR
    cagr = ((end_emissions / start_emissions) ** (1/years_diff)) - 1
    cagr_results[brand] = cagr
    
    # Calculate Total Emissions for the period
    total_emissions = df[brand].sum()
    
    print(f"\nBrand: {brand.split('_CO2e')[0]}")
    print(f"  Start Emissions (2022): {start_emissions:.0f} tCO2e")
    print(f"  End Emissions (2024): {end_emissions:.0f} tCO2e")
    print(f"  Emissions CAGR: {cagr * 100:.2f}%")
    print(f"  Total Emissions (3 yrs): {total_emissions:.0f} tCO2e")

# Calculate Portfolio CAGR
portfolio_start = df.sum(axis=1).iloc[0]
portfolio_end = df.sum(axis=1).iloc[-1]
portfolio_cagr = ((portfolio_end / portfolio_start) ** (1/years_diff)) - 1
print(f"\n--- PORTFOLIO SUMMARY ---")
print(f"  Portfolio CAGR (BAU Baseline): {portfolio_cagr * 100:.2f}%")
print(f"  Portfolio Total Emissions (3 yrs): {df.sum().sum():.0f} tCO2e")
print("----------------------------------------\n")

# --- 4. 10-YEAR PREDICTION MODELING ---

FUTURE_YEARS = 10
FUTURE_INDEX = list(range(df.index.max() + 1, df.index.max() + 1 + FUTURE_YEARS))

# Decarbonization Target (DT): A challenging reduction target (e.g., 7% annual reduction)
ANNUAL_REDUCTION_TARGET = 0.07

# Initialize prediction DataFrames
df_bau = pd.DataFrame(index=FUTURE_INDEX)
df_dt = pd.DataFrame(index=FUTURE_INDEX)

print("--- 10-YEAR EMISSION PREDICTIONS (2025-2034) ---")
print(f"TARGET SCENARIO: Annual Absolute Reduction of {ANNUAL_REDUCTION_TARGET*100:.0f}%\n")

for brand in df.columns:
    last_emissions = df[brand].iloc[-1]
    bau_cagr = cagr_results[brand]
    
    bau_emissions = [last_emissions * (1 + bau_cagr) ** i for i in range(1, FUTURE_YEARS + 1)]
    dt_emissions = [last_emissions * (1 - ANNUAL_REDUCTION_TARGET) ** i for i in range(1, FUTURE_YEARS + 1)]
    
    df_bau[brand] = bau_emissions
    df_dt[brand] = dt_emissions

# Combine historical and future data for visualization/reporting
df_all_bau = pd.concat([df, df_bau])
df_all_dt = pd.concat([df, df_dt])

# Calculate Total Portfolio Emissions for the forecast
total_bau = df_all_bau.sum(axis=1)
total_dt = df_all_dt.sum(axis=1)

print("BUSINESS AS USUAL (BAU) SCENARIO (Growth at Historical CAGR):")
print(f"  Projected Total Portfolio Emissions (2034): {total_bau.iloc[-1]:.0f} tCO2e")
print(f"  Total Cumulative Emissions (10 yrs): {df_bau.sum().sum():.0f} tCO2e")

print("\nDECARBONIZATION TARGET (DT) SCENARIO (7% Annual Reduction):")
print(f"  Projected Total Portfolio Emissions (2034): {total_dt.iloc[-1]:.0f} tCO2e")
print(f"  Total Cumulative Emissions (10 yrs): {df_dt.sum().sum():.0f} tCO2e")

# Calculate CO2e avoided by pursuing DT over BAU
avoided_emissions_cumulative = df_bau.sum().sum() - df_dt.sum().sum()
print(f"\nTOTAL AVOIDED EMISSIONS (DT vs. BAU over 10 years): {avoided_emissions_cumulative:.0f} tCO2e")

# --- 5. REDUCTION STRATEGY RECOMMENDATIONS ---

print("\n\n--- ACTIONABLE REDUCTION STRATEGIES ---")
print(f"\n**INSIGHT 1: The BAU scenario results in {avoided_emissions_cumulative:.0f} tCO2e more emissions over 10 years. Radical intervention is required.**")

print("\n**STRATEGY 1: Material and Manufacturing Decarbonization (Highest Impact)**")
print("Focus: Product-level 'cradle-to-gate' emissions.")
print("Recommendation: Implement a mandatory 'Sustainable Material Score' in sourcing, prioritizing materials that offer at least a 50% reduction in FE.")
print("- Example: EcoThread Apparel could save 30% of its current footprint by shifting 80% of polyester to recycled sources (FE reduction of ~70%).")

print("\n**STRATEGY 2: Last-Mile and Packaging Optimization**")
print("Focus: The consumer experience portion of Scope 3.")
print("Recommendation: Optimize packaging for mass and volume, and switch to regional low-carbon delivery partners.")
print("- Example: TerraHome Goods, due to large item sizes, should reduce packaging weight by 1.5 kg per order, saving ~3.75 kg CO2e (FE * 1.5 kg).")

print("\n**STRATEGY 3: Data Quality and Supplier Engagement**")
print("Focus: Accuracy and systemic change.")
print("Recommendation: Move from industry-average FEs to primary supplier data for the top 20% of high-volume SKUs and high-emission suppliers to accurately track and reward progress.")
print("- Requirement: All Tier 1 suppliers must report on renewable energy consumption within 12 months.")

# --- 6. VISUALIZATION MOCK DATA (For final report/charting) ---
# This data is ready to be exported for a chart showing the two predicted lines.
# Total BAU and DT for reporting:
df_chart = pd.DataFrame({
    'Year': total_bau.index,
    'BAU_Portfolio_CO2e': total_bau.values,
    'DT_Portfolio_CO2e': total_dt.values
})

# Display the final prediction table
print("\n--- PORTFOLIO PREDICTION TABLE ---")
print(df_chart.round(0).tail(11).to_string(index=False))

--- INITIAL DATA OVERVIEW (tonnes CO2e) ---
      EcoThread_Apparel_CO2e  GlowUp_Cosmetics_CO2e  TerraHome_Goods_CO2e
Year                                                                     
2022                    1500                    800                  2200
2023                    1650                    780                  2450
2024                    1850                    750                  2800

--- SIMPLIFIED EMISSION FACTORS USED ---
- Product_Unit_Avg_FE: 20.0 kg CO2e
- Packaging_Mass_FE: 2.5 kg CO2e
- Delivery_FE: 0.8 kg CO2e
----------------------------------------

--- HISTORICAL PERFORMANCE (2022-2024) ---

Brand: EcoThread_Apparel
  Start Emissions (2022): 1500 tCO2e
  End Emissions (2024): 1850 tCO2e
  Emissions CAGR: 11.06%
  Total Emissions (3 yrs): 5000 tCO2e

Brand: GlowUp_Cosmetics
  Start Emissions (2022): 800 tCO2e
  End Emissions (2024): 750 tCO2e
  Emissions CAGR: -3.18%
  Total Emissions (3 yrs): 2330 tCO2e

Brand: TerraHome_Goods
  Start Emissions (2